# Urban Development QnA

https://github.com/simonjisu/UrbanDevQA

## References

* [Anthropic - Building MCP with LLMs](https://modelcontextprotocol.io/tutorials/building-mcp-with-llms)
* [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
* [MCP Servers](https://github.com/modelcontextprotocol/servers)

## Requirements

```
uv, python >= 3.11 
```

* uv: An extremely fast Python package and project manager, written in Rust. - https://docs.astral.sh/uv/guides/projects/

## Installation

Install the following dependencies:

```bash
$ uv venv
$ uv sync
``` 

## Git LFS

We need to install Git LFS (Large File Storage) to handle large files in the repository. After clone this repository, run the following commands:

```bash
$ git lfs install
$ git lfs pull
```

## .env File

It will be helpful to create a `.env` file in the root directory of the project. This file will contain your API keys and other environment variables.

```python
OPENAI_API_KEY="[your_openai_api_key]"
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="[your_langsmith_api_key]"
LANGSMITH_PROJECT="[your_langsmith_project]"
```

* Langsmith is a tool for tracking and analyzing the performance of your LLM applications. It provides a way to log inputs, outputs, and metadata about your LLM calls, which can be useful for debugging and improving your models.
* https://www.langchain.com/langsmith

## Project Structure

```bash
.
├── data
│   ├── geodomain.db            # SQLite database for geodomain
│   ├── LOSA11CD.geojson        # LOSA11CD polygon data
│   ├── NPPF_December_2024.md   # NPPF document
│   └── urban_tasks.json        # questions
├── src
│   ├── __init__.py
│   ├── database.py
│   ├── config.py
│   ├── prompts.py
│   └── servers
│       ├── __init__.py
│       ├── geodomain.py
│       ├── nppf.py
│       └── openstreetmap.py
├── pyproject.toml
├── README.md
├── UrbanDevQA.ipynb
└── uv.lock
```

## Data

### Structure of `geodomain.db`

Tables in the SQLite database:

```
- broadband_speed
- broadband_coverage
- cowz
- cowz_description
- house_age
- house_med_trans
- population_2011
- population_2012
- population_2013
- population_2014
- population_2015
- population_2016
- population_2017
- population_2018
- population_2019
- population_2020
- population_2021
- population_2022
- rmi_base2023
- imd
- spatial_signatures
```

We also have data dictionary for each table in the SQLite database.

```
[dataset_name]-data_dictionary.json
```


###  Output Area

A best fit lookup between 2011 Output Areas (OA) and built-up area sub-divisions (BUASD) and built-up areas (BUA), local authority districts (LAD) and regions (RGN) as at 31 December 2011 in England and Wales. 

In [50]:
import sqlite3
import pandas as pd
from pathlib import Path

data_path = Path('.').resolve() / 'data'

# Read from the geodomain.db
conn = sqlite3.connect(data_path / 'geodomain.db')
with conn:
    query = """SELECT * FROM oa_mosa_lad_rgn LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,OA11CD,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM
0,E00000001,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London
1,E00000003,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London
2,E00000005,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London
3,E00000007,E01000001,City of London 001A,E02000001,City of London 001,E09000001,City of London,E12000007,London
4,E00000010,E01000003,City of London 001C,E02000001,City of London 001,E09000001,City of London,E12000007,London


In [51]:
# visualization of LSOA11CD
import geopandas as gpd
import folium
from IPython.display import display

# Load GeoJSON file
gdf = gpd.read_file('./data/LOSA11CD.geojson')

temp = gdf.loc[gdf['LSOA11NM'].str.contains('London')]

# Get center of the first polygon to center the map
center = temp.geometry[0].centroid
m = folium.Map(location=[center.y, center.x], zoom_start=12)

# Add the GeoDataFrame to the map
for _, row in temp.iterrows():
    # Create popup text or tooltip content
    popup_text = "<br>".join([f"<b>{col}</b>: {row[col]}" for col in gdf.columns if col != 'geometry'])

    folium.GeoJson(
        row["geometry"],
        tooltip=folium.Tooltip(popup_text),
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)
display(m)

### Broadband Speed & Coverage

This data includes the average fixed-line broadband speed by output area, based on 2016-2022 data released by Ofcom. The source data file is linked below. The data is modified from original, I aggregated by LSOA11CD.

* https://data.cdrc.ac.uk/dataset/broadband-speed

In [52]:
with conn:
    query = """SELECT * FROM broadband_speed LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,med_dl_mbps,avg_dl_mbps,min_dl_mbps,max_dl_mbps,avg_dl_lt10_mbps,avg_dl_10to30_mbps,avg_dl_sfbb,avg_dl_ufbb,med_ul_mbps,...,conn_5to10_mbps,conn_10to30_mbps,conn_30to300_mbps,conn_gte300_mbps,conn_gte30_mbps,avg_data_gb,avg_data_lt10_mbps,avg_data_bb,avg_data_sfbb,avg_data_ufbb
0,E01000001,10.45,17.266667,0.3,80.0,7.000000,12.85,57.500000,None,1.1,...,219,264,44,0,44,84.166667,58.0,99.666667,74.500000,None
1,E01000002,10.40,14.060000,0.1,80.0,6.740000,12.58,51.940000,None,1.1,...,260,358,41,0,41,110.600000,70.0,134.400000,197.000000,None
2,E01000003,17.80,25.742857,0.4,80.0,7.933333,15.50,60.685714,None,1.2,...,127,337,118,0,118,143.285714,54.0,130.428571,168.285714,None
3,E01000005,19.65,27.425000,0.1,80.0,7.175000,18.25,60.033333,None,1.2,...,72,559,80,0,80,219.500000,102.0,209.000000,284.000000,None
4,E01000006,43.40,56.880000,3.3,200.0,7.740000,15.22,80.300000,None,6.0,...,29,146,280,0,280,345.400000,244.4,343.400000,353.400000,None


In [53]:
with conn:
    query = """SELECT * FROM broadband_coverage LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,all_premises,all_matched_premises,sfbb_avail_pct_premises,ufbb_avail_pct_premises,full_fibre_avail_pct_premises,pct_no2mbps_premises,pct_no5mbps_premises,pct_no10mbps_premises,pct_no30mbps_premises,...,num_prem_ufbb_avail,num_prem_full_fibre_avail,num_prem_no2mbps,num_prem_no5mbps,num_prem_no10mbps,num_prem_no30mbps,num_prem_below_uso,num_prem_with_nga,num_prem_decent_fwa,num_prem_sfbb_fwa
0,E01000001,1094,1094,75.666667,5.683333,5.400000,0.0,0.0,0.0,18.650000,...,82,80,0,0,0,146,0,948,0,0
1,E01000002,1217,1217,58.266667,16.833333,16.266667,0.0,0.0,0.0,24.900000,...,381,375,0,0,0,123,0,1094,0,0
2,E01000003,1188,1188,21.328571,73.171429,71.842857,0.0,0.0,0.0,5.514286,...,633,618,0,0,0,153,0,1035,0,0
3,E01000005,789,789,7.850000,44.275000,44.275000,0.0,0.0,0.0,47.875000,...,241,241,0,0,0,459,0,330,0,0
4,E01000006,527,527,20.000000,80.000000,0.000000,0.0,0.0,0.0,0.000000,...,409,0,0,0,0,0,0,527,0,0


### House Age / Price / Transaction

Dwelling Ages and Prices data provide Lower Super Output Area (LSOA) level datasets related to residential property dwelling age periods, housing transactions, and median house prices. 

* https://data.cdrc.ac.uk/dataset/dwelling-ages-and-prices

In [54]:
with conn:
    query = """SELECT * FROM house_age LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,dwe_p45pc,dwe_p16pc,dwe_modbp,dwe_mo20bp,dwe_medbp,dwe_mdargb
0,E01020634,0.5105,0.0294,A,A,F,"rgb(117,112,125)"
1,E01020635,0.5348,0.0216,A,A,F,"rgb(118,133,118)"
2,E01020636,0.8000,0.0000,F,F,F,"rgb(90,223,55)"
3,E01020654,0.6793,0.0828,A,X,G,"rgb(140,117,82)"
4,E01020676,0.4951,0.0042,A,A,D,"rgb(83,134,136)"


In [55]:
with conn:
    query = """SELECT * FROM house_med_trans LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,year,quarter,count,median_price
0,E01000001,1995,1,13.0,99500.0
1,E01000002,1995,1,11.0,110000.0
2,E01000003,1995,1,11.0,75000.0
3,E01000005,1995,1,NaN,NaN
4,E01000006,1995,1,4.0,68000.0


### Classification of Workplace Zones (COWZ)

The Classification of Workplace Zones (COWZ- UK) is a UK-wide geodemographic classification of workplaces and the work that is undertaken in them. 

* https://data.cdrc.ac.uk/dataset/classification-workplace-zones-cowz

In [56]:
with conn:
    query = """SELECT * FROM cowz_description LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,level,code,name,colour,penportrait
0,Supergroup,1,Retail,#FEB93D,The retail supergroup is characterised by high...
1,Supergroup,2,Top jobs,#3D76EA,This supergroup is characterised by high densi...
2,Supergroup,3,Metro suburbs,#9731B5,This supergroup is characterised by a higher t...
3,Supergroup,4,Suburban services,#838383,This supergroup is characterised by slightly h...
4,Supergroup,5,Manufacturing and distribution,#8A6F3D,"The manufacturing, transport and distribution ..."


In [57]:
with conn:
    query = """SELECT * FROM cowz LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,WZ11CD,MSOA11CD,LAD11CD,COWZEW_SG,COWZEW_G
0,E33000001,E02006841,E08000020,5,5a
1,E33000002,E02006841,E08000020,7,7a
2,E33000003,E02006841,E08000020,5,5a
3,E33000004,E02006841,E08000020,5,5a
4,E33000005,E02006841,E08000020,7,7b


### Residential Mobility Index

Residential mobility ("Population Churn") is estimated at the household level. Households’ start and end dates are extracted from individual level data by combining individuals that at any point in time have shared time together in the same property or have a shared surname in the same property. First household member determines the ‘start’ date (household identified as moving in), last household member determines the ‘end’ date (household identified as moving out).

* https://data.cdrc.ac.uk/dataset/cdrc-residential-mobility-index

In [58]:
with conn:
    query = """SELECT * FROM rmi_base2023 LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,year,churn
0,E01000001,1997,0.733
1,E01000002,1997,0.792
2,E01000003,1997,0.776
3,E01000005,1997,0.750
4,E01000006,1997,0.779


### Population

Provides annual demographic breakdowns for each LSOA, including total population and gender-specific age bands from 0-9 up to 90+

* https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland

| Key       | Description                          |
|-----------|--------------------------------------|
| `LSOA11CD` | Lower Layer Super Output Area (LSOA) code |
| `Total`    | Total population |
| `F0-9`     | Female population aged 0 to 9 |
| `F10-19`   | Female population aged 10 to 19 |
| `F20-29`   | Female population aged 20 to 29 |
| `F30-39`   | Female population aged 30 to 39 |
| `F40-49`   | Female population aged 40 to 49 |
| `F50-59`   | Female population aged 50 to 59 |
| `F60-69`   | Female population aged 60 to 69 |
| `F70-79`   | Female population aged 70 to 79 |
| `F80-89`   | Female population aged 80 to 89 |
| `F90+`     | Female population aged over 90 |
| `M0-9`     | Male population aged 0 to 9 |
| `M10-19`   | Male population aged 10 to 19 |
| `M20-29`   | Male population aged 20 to 29 |
| `M30-39`   | Male population aged 30 to 39 |
| `M40-49`   | Male population aged 40 to 49 |
| `M50-59`   | Male population aged 50 to 59 |
| `M60-69`   | Male population aged 60 to 69 |
| `M70-79`   | Male population aged 70 to 79 |
| `M80-89`   | Male population aged 80 to 89 |
| `M90+`     | Male population aged over 90 |

In [59]:
with conn:
    query = """SELECT * FROM population_2011 LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,Total,F0-9,F10-19,F20-29,F30-39,F40-49,F50-59,F60-69,F70-79,...,M0-9,M10-19,M20-29,M30-39,M40-49,M50-59,M60-69,M70-79,M80-89,M90+
0,E01000001,1472,46,22,99,110,102,109,96,76,...,40,20,105,155,123,131,112,51,35,4
1,E01000002,1438,65,34,71,97,125,100,99,48,...,37,27,60,142,157,135,110,65,29,8
2,E01000003,1348,38,38,97,104,89,68,73,51,...,34,57,122,145,134,92,68,37,24,5
3,E01000005,987,49,47,101,65,69,45,40,29,...,53,48,118,71,85,67,39,41,7,3
4,E01000006,1731,152,87,168,172,98,77,44,30,...,128,98,184,181,113,83,52,28,13,2


### Index of Multiple Deprivation (IMD)

The Index of Multiple Deprivation (IMD) datasets are small area measures of relative deprivation across each of the constituent nations of the United Kingdom. Areas are ranked from the most deprived area (rank 1) to the least deprived area. Each nation publishes its data on its own data portal. Each nation measures deprivation in a slightly different way but the broad themes include income, employment, education, health, crime, barriers to housing and services, and the living environment.

* https://data.cdrc.ac.uk/dataset/index-multiple-deprivation-imd

| Key          | Description |
|--------------|-------------|
| `LSOA11CD`   | Lower Layer Super Output Area (LSOA) code |
| `LAD11CD`    | Local Authority District (LAD) code |
| `Rank`       | A simple ranking of the SOA from most to least deprived. |
| `SOA_pct`    | Percentage - where the most deprived 1% of SOAs are 1 and the next most deprived 1% are 2 etc. |
| `SOA_decile` | Grouping the most deprived 10% of SOAs as Decile 1 and the second most deprived 10% as decile 2 etc. |
| `LA_Rank`    | A simple ranking of the LA from most to least deprived. |
| `LA_pct`     | Percentage - where the most deprived 1% of LAs are 1 and the next most deprived 1% are 2 etc. |
| `LA_decile`  | Grouping the most deprived 10% of LAs as Decile 1 and the second most deprived 10% as decile 2 etc. |

* SOA - Super Output Area
* LA - Local Authority

#### IMD 2019 Domains 
The IMD2019 is comprised of seven distinct domains of deprivation which, when combined and appropriately weighted, form the IMD2019. They are;
- Income (22.5%)
- Employment (22.5%)
- Health Deprivation and Disability (13.5%)
- Education, Skills Training (13.5%)
- Crime (9.3%)
- Barriers to Housing and Services (9.3%)
- Living Environment (9.3%)

In [60]:
with conn:
    query = """SELECT * FROM imd LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,Rank,SOA_pct,SOA_decile,LA_Rank,LA_pct,LA_decile,LAD11CD
0,E01000001,29199,89,9,208,66,7,E09000001
1,E01000002,30379,92,10,208,66,7,E09000001
2,E01000003,14915,45,5,208,66,7,E09000001
3,E01000005,8678,26,3,208,66,7,E09000001
4,E01000006,14486,44,5,5,2,1,E09000002


### Spatial Signatures of Great Britain

Geographical Characterisation of British Urban Form and Function using the Spatial Signatures Framework

* https://data.cdrc.ac.uk/dataset/spatial-signatures-great-britain
* https://urbangrammarai.xyz/great-britain/

| Key          | Description |
|---------------|-------------|
| `LSOA11CD`    | Lower Layer Super Output Area (LSOA) code |
| `primary_code` | Primary spatial signature code |
| `primary_type` | Primary spatial signature type (name) |
| `COA` | Countryside agriculture. Features much of the English countryside and displays a high degree of agriculture including both fields and pastures. There are a few buildings scattered across the area but, for the most part, it is green space. |
| `ACS` | Accessible suburbia. Residential development on the urban periphery with a relatively legible and connected street network. Low density, lack of jobs and services. Mostly dormitory areas. |
| `OPS` | Open sprawl. Transition between countryside and urbanised land, with large open spaces and mixed developments from highways to neighborhoods. |
| `WIC` | Wild countryside. Least human influence, large open spaces, occasional roads or cottages, found in places like the Scottish Highlands and national parks. |
| `WAL` | Warehouse/Park land. Industrial or work-related developments, large buildings, many manual jobs, and low population. May include parks. |
| `GRQ` | Gridded residential quarters. Grid-like street patterns with small blocks, residential focus, some services and jobs. Located away from city centers. |
| `URB` | Urban buffer. Green belt areas adjacent to towns or cities, mostly agricultural but smaller than other countryside types. |
| `DIS` | Disconnected suburbia. Outskirts developments with convoluted street networks, low density, lacking services, car-dependent. |
| `DRN` | Dense residential neighbourhoods. High population density, residential focus, varied street patterns, some services and jobs. |
| `CRN` | Connected residential neighbourhoods. Dense urban areas with good street connectivity, access to services, and jobs. Often outside city centers. |
| `DUN` | Dense urban neighbourhoods. Inner-city areas with high population and built-up density, residential but with direct access to services and jobs. Walkable. |
| `LOU` | Local urbanity. Town or district centers, high density, good access to amenities and jobs, but with only local significance. |
| `DIU` | Concentrated urbanity. Found only in central London, with high job and amenity density, complex building shapes, and little green space. |
| `REU` | Regional urbanity. Centers of mid-size cities like Liverpool or Newcastle, high density and walkable access to jobs and amenities. |
| `MEU` | Metropolitan urbanity. Core of large cities like Glasgow or Manchester, very high job and built-up density. |
| `HDU` | Hyper concentrated urbanity. Found only in central London (e.g., Soho), extremely dense with rich historical layering and abundant amenities. |
| `OUT` | Others |


In [61]:
with conn:
    query = """SELECT * FROM spatial_signatures LIMIT 5"""
    df = pd.read_sql(query, conn)
df.head()

,LSOA11CD,primary_code,primary_type,COA,ACS,OPS,WIC,WAL,GRQ,URB,DIS,DRN,CRN,DUN,LOU,DIU,REU,OUT,MEU,HDU
0,E01000007,DUN,Dense urban neighbourhoods,0.0,0.0000,0.0000,0.0,0.0000,0.000,0.0,0.0000,0.0000,0.0000,0.8227,0.1773,0.0,0.0,0.0000,0.0,0.0
1,E01000015,DRN,Dense residential neighbourhoods,0.0,0.0000,0.0011,0.0,0.0000,0.000,0.0,0.0228,0.7076,0.1368,0.1303,0.0000,0.0,0.0,0.0014,0.0,0.0
2,E01000030,DRN,Dense residential neighbourhoods,0.0,0.0000,0.0000,0.0,0.4277,0.000,0.0,0.0000,0.5723,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0
3,E01000085,DUN,Dense urban neighbourhoods,0.0,0.0000,0.0000,0.0,0.0000,0.000,0.0,0.0000,0.1718,0.1266,0.7016,0.0000,0.0,0.0,0.0000,0.0,0.0
4,E01000118,CRN,Connected residential neighbourhoods,0.0,0.3397,0.0362,0.0,0.0000,0.132,0.0,0.0000,0.0043,0.4878,0.0000,0.0000,0.0,0.0,0.0000,0.0,0.0


### Points of Interest

This dataset contains Point of Interest (POI) data for the United Kingdom, obtained from the Overture Maps Foundation. While the Overture Maps Foundation provides a series of global datasets, this CDRC data product provides users with easy access, without having to query the AWS hosted data that Overture Maps Foundation provide. 

* https://data.cdrc.ac.uk/dataset/point-interest-data-united-kingdom

In [62]:
with conn:
    query = """SELECT * FROM poi WHERE LSOA11CD = "E01000001" """
    df = pd.read_sql(query, conn)
print(df.shape)
df.head()

(219, 12)


,primary_name,main_category,alternate_category,address,locality,postcode,region,country,source,lat,long,LSOA11CD
0,Barts Heart Centre,hospital,health_and_medical|medical_center,St Bartholomew's Hospital,None,None,None,GB,meta,51.517158,-0.099490,E01000001
1,BT.shop,shopping,business,81 Newgate Street,London,EC1A 7AJ,ENG,GB,meta,51.515668,-0.097898,E01000001
2,Baby Birth Facts,shopping,None,None,None,None,None,GB,meta,51.515988,-0.097805,E01000001
3,London St Paul’s,transportation,commercial_real_estate|pub,5 Cheapside,London,EC2V 6AA,ENG,GB,meta,51.515158,-0.097188,E01000001
4,Coco di Mama - Italian To Go - Cheapside,coffee_shop,food|restaurant,10 St Martins-Le-Grand,London,EC1A 4EN,None,GB,msft,51.515082,-0.097032,E01000001


In [63]:
# Filter the GeoDataFrame to include only the LSOA11CDs present in the POI data
lsoa_polygon = gdf.loc[gdf['LSOA11CD'].isin(df['LSOA11CD'].values.tolist())]

# Get center of the first polygon to center the map
center = lsoa_polygon['geometry'].iloc[0].centroid
m = folium.Map(location=[center.y, center.x], zoom_start=16)
name = lsoa_polygon['LSOA11NM'].iloc[0]
popup_text = "<br>".join([f"<b>{col}</b>: {lsoa_polygon[col].iloc[0]}" for col in lsoa_polygon.columns if col != 'geometry'])

folium.GeoJson(
    lsoa_polygon["geometry"].iloc[0],
    popup=folium.Popup(popup_text, max_width=300),
).add_to(m)

# Add the GeoDataFrame to the map
for _, row in df.sample(30).iterrows():
    # Create popup text or tooltip content
    lat = row['lat']
    lon = row['long']
    popup_text = "<br>".join([f"<b>{col}</b>: {row[col]}" for col in ['primary_name', 'main_category']])
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='blue', icon='info-sign'),
        popup=folium.Popup(popup_text, max_width=300),
    ).add_to(m)
display(m)

---

# MCP Servers

We have three servers.

1. **Geodomain**: This server provides access to the geodomain database, which contains various datasets related to urban development and demographics.
2. **NPPF**: This server provides access to the National Planning Policy Framework (NPPF) data, which is used for planning and development purposes.
3. **OpenStreetMap**: This server provides access to OpenStreetMap data, which is a collaborative mapping project that provides free geographic data and mapping to anyone who wants to use it.

## Start MCP Server

```bash
# start at the root of the project
# /home/[user]/UrbanDevQA
$ pwd

# start the environment if not already started 
# assume 
$ source ./venv/bin/activate

# start servers (run these code on 3 different terminals)
(UrbanDevQA) $ uv run -m src.servers.geodomain
(UrbanDevQA) $ uv run -m src.servers.nppf
(UrbanDevQA) $ uv run -m src.servers.openstreetmap
```

Once the servers are started, you can keep continue to run following cells in the notebook.:

In [64]:
from dotenv import load_dotenv, find_dotenv
from src.config import ServersConfig

_ = load_dotenv(find_dotenv())

# Load the configuration
CONFIG = ServersConfig()
print(CONFIG.servers_path)

from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "geodomain": {
            "url": "http://localhost:8111/sse",
            "transport": "sse",
        },
        "openstreetmap": {
            "url": "http://localhost:8112/sse",
            "transport": "sse",
        },
        "nppf": {
            "url": "http://localhost:8113/sse",
            "transport": "sse",
        },
    }
)

await client.__aenter__()

/Users/sungjuyong/Documents/Causality_lab/Class/1/Topics/UrbanDevQA/src/servers


In [65]:
subquery_client = MultiServerMCPClient(
    {
        "sequential_thinking": {
            "url": "http://localhost:8114/sse",
            "transport": "sse",
        }
    }
)

await subquery_client.__aenter__()

## Query Decomposition

In [66]:
from pydantic import BaseModel, Field, ConfigDict
from src.prompts import Prompts
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI


# # query
# # query = """What is a good place to start a fast-food restaurant at the Southwark of London?
# # Please meet the following requirements:
# # 1. meet the regulation
# # 2. have a good population traffic
# # """
# query = """Where would be a suitable spot to build a new primary school in Westminster? 
# Focus on areas zoned for education, with a high number of children, and where existing schools aren't overcrowded.
# """

# # define structural output for query decomposition/merge
# class Output(BaseModel):
#     """Subqueries for the query analysis"""
#     subqueries: list[str] = Field(description="List of subqueries")

# # define the model
# model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.8)
# structured_llm = model.with_structured_output(Output)
# prompt_decompose = ChatPromptTemplate.from_messages(
#     [
#         ("system", Prompts.query_decomposition),
#         ("human", "{query}"),
#     ]
# )
# query_decompistion = {"query": RunnablePassthrough()} | prompt_decompose | structured_llm 
# output: Output = query_decompistion.invoke({"query": query})

In [67]:
from langgraph.prebuilt import create_react_agent
from typing import List
from pydantic import BaseModel

query = """Where would be a suitable spot to build a new primary school in Westminster? 
Focus on areas zoned for education, with a high number of children, and where existing schools aren't overcrowded.
"""

class SubQuery(BaseModel):
    query: str
    data_type: str

class Output(BaseModel):
    """Subqueries for the query analysis"""
    subqueries: list[str] = Field(description="List of subqueries")
    
# 모델 준비
model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.8)

# subquery_client에서 tools 추출
tools = subquery_client.get_tools()

# 에이전트 생성
subquery_executor_agent = create_react_agent(
    model=model,
    tools=tools,
    prompt="You are an expert agent that uses tools to answer structured subqueries."
)

async def execute_subqueries_with_agent(subqueries: List[SubQuery]):
    results = {}
    for sub in subqueries:
        print(f"\n▶ Running agent on subquery: {sub.query}")
        try:
            result = await subquery_executor_agent.ainvoke({"input": sub.query})
            results[sub.query] = result
            print(f"✅ Result: {result}")
        except Exception as e:
            results[sub.query] = f"❌ Error: {str(e)}"
            print(f"❌ Error: {e}")
    return results

def get_query_decomposer(model_name: str = "gpt-4.1-nano", temperature: float = 0.8):
    """
    Returns a runnable pipeline for decomposing a complex query into structured subqueries.
    """
    model = ChatOpenAI(model=model_name, temperature=temperature)
    structured_llm = model.with_structured_output(Output)
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", Prompts.query_decomposition),
            ("human", "{query}"),
        ]
    )
    
    return {"query": RunnablePassthrough()} | prompt | structured_llm

if __name__ == "__main__":
    decomposer = get_query_decomposer()
    output: Output = decomposer.invoke({"query": query})

    # print("🔍 Decomposed Subqueries:")
    # for s in output.subqueries:
    #     print(f"- {s.query} ({s.data_type})")
    print(output.subqueries)

    # async 환경에서 실행
    # import asyncio

    # async def run_all():
    #     async with subquery_client:
    #         tool_results = await execute_subqueries_with_agent(output.subqueries)
    #         print("\n📌 Final Tool Results:")
    #         for q, res in tool_results.items():
    #             print(f"{q}\n  ↳ {res}")

    # asyncio.run(run_all())

['Identify the geographic boundaries of Westminster to define the study area.', 'Determine zones within Westminster that are designated or zoned for educational facilities.', 'Gather demographic data on the number of children (aged relevant to primary school age) in different neighborhoods of Westminster.', 'Identify existing primary schools within Westminster and assess their current capacity and level of overcrowding.', 'Map the locations of current schools against demographic data to identify areas with high child populations and insufficient school capacity.', 'Assess transportation infrastructure and accessibility to potential sites for ease of student commute.', 'Evaluate environmental and land use policies that may impact the suitability of potential sites for new schools.', 'Review any local planning policies, zoning restrictions, or community consultations pertaining to new school development.', 'Analyze socioeconomic data to understand community needs and potential support or

In [68]:
from pydantic import BaseModel, Field
from src.prompts import Prompts
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from typing import List, Optional
from enum import Enum

# class DataType(str, Enum):
#     GEOSPATIAL = "geospatial"
#     DEMOGRAPHIC = "demographic"
#     POLICY = "policy"
#     INFRASTRUCTURE = "infrastructure"
#     ENVIRONMENTAL = "environmental"
#     ECONOMIC = "economic"
#     SOCIAL = "social"

# class PriorityLevel(str, Enum):
#     HIGH = "high"
#     MEDIUM = "medium"
#     LOW = "low"

# class SubQuery(BaseModel):
#     """Individual subquery with metadata"""
#     query: str = Field(description="The actual subquery text")
#     data_type: DataType = Field(description="Type of data needed to answer this query")
#     priority: PriorityLevel = Field(description="Priority level of this subquery")
#     dependencies: List[str] = Field(default_factory=list, description="List of other subquery IDs this depends on")
#     description: str = Field(description="Brief description of what this subquery aims to answer")

# class Output(BaseModel):
#     """Structured output for query decomposition"""
#     subqueries: List[SubQuery] = Field(description="List of subqueries with metadata")
#     main_components: List[str] = Field(description="Main components identified in the original query")
#     summary: str = Field(description="Brief summary of how the subqueries work together to answer the original query")

# def get_query_decomposer(model_name: str = "gpt-4.1-nano", temperature: float = 0.8):
#     """
#     Returns a runnable pipeline for decomposing a complex query into structured subqueries.
#     """
#     model = ChatOpenAI(model=model_name, temperature=temperature)
#     structured_llm = model.with_structured_output(Output)
    
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", Prompts.query_decomposition),
#             ("human", "{query}"),
#         ]
#     )
    
#     return {"query": RunnablePassthrough()} | prompt | structured_llm

# if __name__ == "__main__":
#     # Example query
#     query = """Where would be a suitable spot to build a new primary school in Westminster? 
#     Focus on areas zoned for education, with a high number of children, and where existing schools aren't overcrowded.
#     """

#     # Run decomposition
#     query_decomposition = get_query_decomposer()
#     output: Output = query_decomposition.invoke({"query": query})
    
    
#     print(output.subqueries)
    
# subquery_list = [sub.query for sub in output.subqueries]

[SubQuery(query='Identify the geographic boundaries of Westminster to focus the search area.', data_type=<DataType.GEOSPATIAL: 'geospatial'>, priority=<PriorityLevel.HIGH: 'high'>, dependencies=[], description='Determine the spatial extent of Westminster for localized analysis.'), SubQuery(query='Locate zones within Westminster that are designated for educational use or have zoning policies supportive of school construction.', data_type=<DataType.POLICY: 'policy'>, priority=<PriorityLevel.HIGH: 'high'>, dependencies=['Identify the geographic boundaries of Westminster to focus the search area.'], description='Find areas with zoning regulations that permit or favor building new schools.'), SubQuery(query='Obtain demographic data on the number of children and school-age population in different neighborhoods within Westminster.', data_type=<DataType.DEMOGRAPHIC: 'demographic'>, priority=<PriorityLevel.HIGH: 'high'>, dependencies=['Identify the geographic boundaries of Westminster to focus 

In [69]:
sub_queries = output.subqueries
subquery_list = [sub.split("(")[0].strip() for sub in output.subqueries]
subquery_list

AttributeError: 'SubQuery' object has no attribute 'split'

## Getting answers for subqueries with ReAct agent 

* https://arxiv.org/abs/2210.03629

### ReAct Prompt

```
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
```

In [41]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig
from datetime import datetime
from langchain_teddynote.messages import astream_graph
from typing import Optional
from src.prompts import Prompts

# define the structural output for the answer
class AnswerWithQuotes(BaseModel):
    """output with references"""
    answer: str = Field(description="Part of the answer to the question")
    references: Optional[list[str]] = Field(description="List of references")

class AnswerList(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    answerlist: list[AnswerWithQuotes] = Field(
        description="List of answers with references"
    )

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0.5)

prompt = Prompts.geodomain.format(
    today=datetime.now().strftime("%Y-%m-%d")
)

agent = create_react_agent(
    model, tools=client.get_tools(), prompt=prompt, 
    response_format=AnswerList
)

In [ ]:
sub_answers = []

def pretty_print(answer: AnswerList):
    for a in answer.answerlist:
        print(f"{a.answer}")
    if a.references:
        for ref in a.references:
            print(f"Reference: {ref}")
    print("===============")

for i, subquery in enumerate(subquery_list):
    print(f'[Subquery {i}] {subquery}')
    inputs = {"messages": [("user", subquery)]}
    config = RunnableConfig(recursion_limit=60, thread_id=i)
    answer = await agent.ainvoke(inputs, config=config)
    sub_answers.append(answer)
    pretty_print(answer['structured_response'])

[Subquery 0] What are the geographic boundaries and zoning regulations for Westminster to identify areas designated for educational use?
Westminster's geographic boundaries correspond to the local authority district code E09000033. Within this area, several locations are designated for educational use including institutions such as Imperial College Adult Education, the Royal College of Art, and other educational facilities located in specific Lower Layer Super Output Areas (LSOAs). Zoning regulations for educational use are guided by the National Planning Policy Framework (NPPF), which emphasizes efficient land use, sustainable development, and the provision of appropriate facilities including educational buildings. Local planning policies align with these national guidelines to designate land for educational purposes while ensuring optimal site use and protection of the environment.
Reference: Local Authority District Code for Westminster: E09000033
Reference: Points of Interest data 

## Getting final answer 

In [43]:
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate


class FinalAnswer(BaseModel):
    """Final answer"""
    answer: str = Field(description="Final answer to the question")

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0.5)
structured_llm = model.with_structured_output(FinalAnswer)

# prepare the sub_answer_str
sub_answer_str = ""
for sub_q, sub_as in zip(sub_queries, sub_answers):
    for sub_a in sub_as['structured_response'].answerlist:
        sub_answer_str += f"Sub Query: {sub_q}\n"
        sub_answer_str += f"Sub Answer: {sub_a.answer}\n"
        if sub_a.references:
            for ref in sub_a.references:
                sub_answer_str += f"Reference: {ref}\n"
    sub_answer_str += "================\n"

prompt_template = """You are an assistant that can answer questions using structured findings.

# Original Question:
{question}

# Subquery Findings:
You investigated the question by asking the following subqueries and received these responses:
{sub_answer_str}

---

# Task:
Based on the information provided:
- Identify and select **only one most suitable location** as the answer to the question.
- Justify why this specific location is the best choice using evidence from the subquery results.
- Do not include multiple candidates or vague recommendations.
- Avoid making assumptions not supported by the findings.

---

# Output format
You should be in the following format with JSON format:
```
FinalAnswer:
- answer: <str, Final answer to the question>
```
"""

prompt = PromptTemplate.from_template(prompt_template)

chain: RunnableSequence = (prompt | structured_llm)
final_answer = chain.invoke(
    {
        "question": query,
        "sub_answer_str": sub_answer_str
    }
)

In [44]:
# from pprint import pprint
# pprint(final_answer.answer)
print(final_answer.answer)

The most suitable location to build a new primary school in Westminster is within Lower Layer Super Output Area (LSOA) E01023155. This area is identified as having a high population of children but is currently underserved by existing primary schools, as evidenced by zero primary school count in this LSOA. Additionally, the location falls within Westminster's geographic boundaries (E09000033) and aligns with zoning regulations that designate land for educational use. Building a new school here would address the unmet demand for primary education in a high-need area without contributing to overcrowding in existing schools. Environmental and infrastructure constraints must be managed according to local planning policies, but the proactive land use planning framework in Westminster supports development in such designated zones to meet community needs effectively.


In [ ]:
from langchain.schema import BaseMessage
from typing import Tuple

# 평가용 모델 정의
llm_as_a_judge = ChatOpenAI(model="gpt-4.1-mini", temperature=0.5)

# 평가 프롬프트 템플릿
evaluation_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a fair but strict evaluator. Your job is to rate an AI-generated answer to a query on a scale from 1 to 5.\n"
     "Evaluate the answer based on how well it satisfies the query, considering relevance, clarity, accuracy, and completeness.\n"
     "Please use the proper server given user's question as much as possible.\n"
     "If the answer requires geospatial data, prefer: A. Geospatial domain.\n"
     "If the answer requires map data, prefer: B. OpenStreetMap.\n"
     "If the answer involves planning or zoning policy, prefer: C. National Policy Planning Framework (NPPF)."
    ),
    ("human", 
     "Query:\n{query}\n\n"
     "Answer:\n{answer}\n\n"
     "Evaluate the answer. Return only:\n"
     "Score: <1-5>\n"
     "Reason: <brief explanation>")
])

# 평가 함수
def judge_answer(query: str, answer: str) -> Tuple[int, str]:
    messages: list[BaseMessage] = evaluation_prompt.format_messages(query=query, answer=answer)
    response = llm_as_a_judge(messages)
    
    # 간단한 파싱
    lines = response.content.strip().split('\n')
    score_line = next((line for line in lines if "score" in line.lower()), "")
    score = int(''.join(filter(str.isdigit, score_line))) if score_line else 0
    reason_lines = [line for line in lines if "reason" in line.lower() or not line.lower().startswith("score")]
    reason = "\n".join(reason_lines).replace("Reason:", "").strip()
    
    return score, reason

# ─── 평가 실행 ───
score, reason = judge_answer(query, answer)

# ─── 결과 출력 ───
print(f"Score: {score}/5")
print(f"Reason: {reason}")

Score: 4/5
Reason: The answer identifies a specific area within Westminster suitable for a new primary school based on high child population and lack of existing schools, addressing the query well. It also notes zoning compliance and planning considerations. However, it lacks detailed data on existing school overcrowding and does not provide explicit references to zoning maps or planning frameworks, which would enhance completeness and accuracy.
